In [38]:
import featuretools as ft
import pandas as pad
import numpy as np
import os

lc_data=pad.read_csv("loan.csv", low_memory=False)


In [39]:
#Assigning variables to target field i.e interest rates and different id's

interest_rate=lc_data['int_rate']
id=lc_data['id']
member_id=lc_data['member_id']

#lc_data.drop(['int_rate'], axis=1, inplace=True)

In [40]:
#Merging id and member_id fields

lc_data['uid']=lc_data['id']+lc_data['member_id']
lc_data.drop(['id'], axis=1, inplace=True)
#lc_data.drop(['member_id'], axis=1, inplace=True)

In [41]:
es=ft.EntitySet(id='interest_rate')

es.entity_from_dataframe(entity_id = 'credit_rating', dataframe = lc_data, index = 'id')

es.normalize_entity(base_entity_id='credit_rating', new_entity_id='borrower', index = 'member_id', 
additional_variables = ['addr_state','annual_inc','annual_inc_joint','application_type', 'desc', 'emp_length', 'home_ownership', 'verification_status_joint'])


2019-03-21 02:18:24,049 featuretools.entityset - WARNING    index id not found in dataframe, creating new integer column


Entityset: interest_rate
  Entities:
    credit_rating [Rows: 379976, Columns: 34]
    borrower [Rows: 379976, Columns: 9]
  Relationships:
    credit_rating.member_id -> borrower.member_id

In [42]:
#data preprocessing
print(es)

Entityset: interest_rate
  Entities:
    credit_rating [Rows: 379976, Columns: 34]
    borrower [Rows: 379976, Columns: 9]
  Relationships:
    credit_rating.member_id -> borrower.member_id


In [43]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'credit_rating', 
max_depth = 1, 
verbose = 1, 
n_jobs = 4)

Built 40 features
EntitySet scattered to workers in 26.626 seconds

Elapsed: 00:00 | Remaining: ? | Progress:   0%|          | Calculated: 0/11 chunks
Elapsed: 00:02 | Remaining: 00:25 | Progress:   9%|▉         | Calculated: 1/11 chunks
Elapsed: 00:03 | Remaining: 00:18 | Progress:  18%|█▊        | Calculated: 2/11 chunks
Elapsed: 00:04 | Remaining: 00:14 | Progress:  27%|██▋       | Calculated: 3/11 chunks
Elapsed: 00:05 | Remaining: 00:08 | Progress:  45%|████▌     | Calculated: 5/11 chunks
Elapsed: 00:06 | Remaining: 00:06 | Progress:  55%|█████▍    | Calculated: 6/11 chunks
Elapsed: 00:06 | Remaining: 00:00 | Progress:  91%|█████████ | Calculated: 10/11 chunks
Elapsed: 00:07 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


tornado.application - ERROR - Exception in Future <Future cancelled> after timeout
Traceback (most recent call last):
  File "C:\Users\adity\Anaconda3\lib\site-packages\tornado\gen.py", line 970, in error_callback
    future.result()
concurrent.futures._base.CancelledError
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:53530 remote=tcp://127.0.0.1:53491>


In [46]:
feature_matrix = feature_matrix.reindex(index=lc_data['id'])
feature_matrix = feature_matrix.reset_index()


ValueError: cannot reindex from a duplicate axis

In [47]:
feature_matrix.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,verification_status,...,open_il_12m,open_il_24m,uid,borrower.addr_state,borrower.annual_inc,borrower.annual_inc_joint,borrower.application_type,borrower.emp_length,borrower.home_ownership,borrower.verification_status_joint
uid,,,,,,,,,,,,,,,,,,,,,
2374100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2391597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2390699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2354041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2387106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
